https://docs.microsoft.com/pt-br/archive/msdn-magazine/2008/january/sql-server-uncover-hidden-data-to-optimize-application-performance

Many application performance problems can be traced to poorly performing database queries; however, there are many ways you can improve database performance. SQL ServerTM 2005 gathers a lot of information that you can use to identify the causes of such performance issues.

SQL Server 2005 collects data relating to running queries. This data, which is held in memory and starts accumulating after a server restart, can be used to identify numerous issues and metrics, including those surrounding table indexes, query performance, and server I/O. You can query this data via the SQL Server Dynamic Management Views (DMVs) and related Dynamic Management Functions (DMFs). These are system-based views and functions that present server state information that can be used to diagnose problems and tune database performance.

In this article, I will highlight areas where performance can be improved by using information that is already being gathered by SQL Server 2005. This approach is largely non-intrusive, as it collects and examines existing data, typically querying underlying system data.

I will demonstrate how to obtain this information, discuss the underlying DMVs, identify any caveats to be aware of when interpreting the data, and point you to additional areas where you might be able to realize performance improvements. To do this, I will present and examine a series of SQL scripts that detail the various aspects of the data collected by SQL Server 2005. A complete and fully commented version of this script can be downloaded from the MSDN® Magazine Web site.

Some of the steps I will discuss concentrate on the server as a whole, including all the databases hosted on a given server. When necessary, it is possible to concentrate on a given database by adding the appropriate filtering, such as adding its name to the query.

Conversely, some of the queries join to the sys.indexes DMV, which is a database-specific view that reports results only for the current database. In these cases, I amended the queries to iterate over all the databases on the server by using the system stored procedure sp\_MSForEachDB, thus presenting server-wide results.

In order to target the most relevant records for a given performance metric, I will limit the number of records returned using the SQL TOP function.

## **Causes of Server Waits**

A user typically experiences poor performance as a series of waits. Whenever a SQL query is able to run but is waiting on another resource, it records details about the cause of the wait. These details can be accessed using the sys.dm\_os\_wait\_stats DMV. You can examine the accumulated cause of all the waits using the SQL script

In [1]:
SELECT
    TOP 10
    [Wait type] = wait_type
    ,[Wait time (s)] = wait_time_ms / 1000
    ,[% waiting] = CONVERT(DECIMAL(12,2), wait_time_ms * 100.0 / SUM(wait_time_ms) OVER())
FROM
    sys.dm_os_wait_stats
WHERE wait_type NOT LIKE '%SLEEP%'
ORDER BY wait_time_ms DESC;

(10 rows affected)

Total execution time: 00:00:19.391

Wait type,Wait time (s),% waiting
HADR_WORK_QUEUE,37745082,"49,07"
BROKER_TASK_STOP,4754907,"6,18"
CLR_AUTO_EVENT,3600558,"4,68"
HADR_NOTIFICATION_DEQUEUE,3599878,"4,68"
BROKER_TRANSMITTER,3599084,"4,68"
DISPATCHER_QUEUE_SEMAPHORE,1874467,"2,44"
HADR_CLUSAPI_CALL,1801359,"2,34"
HADR_FILESTREAM_IOMGR_IOCOMPLETION,1800277,"2,34"
DIRTY_PAGE_POLL,1800194,"2,34"
REQUEST_FOR_DEADLOCK_SEARCH,1800174,"2,34"


## **Reads and Writes**

High I/O usage can be an indicator of poor data access mechanisms. SQL Server 2005 keeps track of the total number of reads and writes that each query uses to fulfill its needs. You can sum these numbers to determine which databases perform the most overall reads and writes.

The sys.dm\_exec\_query\_stats DMV contains aggregate performance statistics for cached query plans. This includes information about the number of logical reads and writes and the number of times the query has executed. When you join this DMV to the sys.dm\_exec\_sql\_text DMF, you can sum the number of reads and writes by database. Notice that I use the new SQL Server 2005 CROSS APPLY operator to handle this join. The script I use to identify which databases are using the most reads and writes

In [2]:
SELECT
    TOP 10
    [Total Reads] = SUM(total_logical_reads)
    ,[Execution count] = SUM(qs.execution_count)
    ,DatabaseName = DB_NAME(qt.dbid)
FROM
    sys.dm_exec_query_stats qs 
    CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) AS qt
GROUP BY DB_NAME(qt.dbid)
ORDER BY [Total Reads] DESC;



SELECT
    TOP 10
    [Total Writes] = SUM(total_logical_writes)
    ,[Execution count] = SUM(qs.execution_count)
    ,DatabaseName = DB_NAME(qt.dbid)
FROM
    sys.dm_exec_query_stats qs
     CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) AS qt
GROUP BY DB_NAME(qt.dbid)
ORDER BY [Total Writes] DESC;

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:01.527

Total Reads,Execution count,DatabaseName
33879587301,28088071,NULL
1577500819,178633,Integracao_BlackBoard
394126300,137383,Senac_Producao
162790692,2792,CRM
114780077,1732613,msdb
81824024,32,SESC_SENAC
28243100,79,PIMv2
15704441,85499,Wifi
11300389,1843436,DBMon
5031493,1475,PainelEAD


Total Writes,Execution count,DatabaseName
18758573,28088084,NULL
3554377,178633,Integracao_BlackBoard
3086352,1732613,msdb
1315170,137383,Senac_Producao
313699,79,PIMv2
168177,2792,CRM
99964,32,SESC_SENAC
74054,85499,Wifi
52722,1475,PainelEAD
20835,1843436,DBMon


## **Missing Indexes by Database**

When SQL Server processes a query, the optimizer keeps a record of the indexes it attempts to use to satisfy the query. If these indexes are not found, SQL Server creates a record of the missing index. This information can be viewed using the sys.dm\_db\_missing\_index\_details DMV.

You can show which databases on a given server are missing indexes using the script shown in Figure 3. Discovering these missing indexes is important because the indexes will often provide an ideal path to retrieving query data. In turn, this can reduce I/O and improve overall performance. My script examines sys.dm\_db\_missing\_index\_details and sums up the number of missing indexes per database, making it easy to determine which databases need further investigation.

In [3]:
SELECT
    DatabaseName = DB_NAME(database_id) 
    ,[Number Indexes Missing] = count(*)
FROM
    sys.dm_db_missing_index_details
GROUP BY DB_NAME(database_id)
ORDER BY 2 DESC;

(9 rows affected)

Total execution time: 00:00:00.038

DatabaseName,Number Indexes Missing
CRM,235
PainelEAD,122
Integracao_BlackBoard,73
PIMv2,16
Senac_Producao,14
Integrador,5
msdb,4
Wifi,2
PainelGestao,1


## **Costly Missing Indexes**

[https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-db-missing-index-group-stats-transact-sql?view=sql-server-ver15](https://docs.microsoft.com/pt-br/sql/relational-databases/system-dynamic-management-views/sys-dm-db-missing-index-group-stats-transact-sql?view=sql-server-ver15)  

Indexes will have varying levels of impact on query performance. You can drill down into the most costly missing indexes across all the databases on the server, finding out which missing indexes are likely to have the most significant positive impact on performance. The sys.dm\_db\_missing\_index\_group\_stats DMV notes the number of times SQL has attempted to use a particular missing index. The sys.dm\_db\_missing\_index\_details DMV details the missing index structure, such as the columns required by the query. These two DMVs are linked together via the sys.dm\_db\_missing\_index\_groups DMV. The cost of the missing index (the total cost column) is calculated as the product of the average total user cost and the average user impact multiplied by the sum of the user seeks and user scans. You can use the script shown in Figure 4 to identify the most costly missing indexes. The results of this query, which are ordered by Total Cost, show the cost of the most important missing indexes along with details about the database/schema/table and the columns required in the missing indexes. Specifically, this script identifies which columns are used in equality and inequality SQL statements. Additionally, it reports which other columns should be used as included columns in a missing index. Included columns allow you to satisfy more covered queries without obtaining the data from the underlying page, thus using fewer I/O operations and improving performance.

In [2]:
SELECT
    TOP 10
    [Total Cost] = ROUND((avg_total_user_cost * avg_user_impact * (user_seeks + user_scans))/100,0) 
    ,avg_user_impact /*porcentagem media de melhoria por consulta*/
    ,(user_seeks + user_scans) query_count
    ,TableName = statement 
    ,[EqualityUsage] = equality_columns 
    ,[InequalityUsage] = inequality_columns 
    ,[Include Cloumns] = included_columns
FROM
    sys.dm_db_missing_index_groups g
    INNER JOIN sys.dm_db_missing_index_group_stats s ON s.group_handle = g.index_group_handle
    INNER JOIN sys.dm_db_missing_index_details d ON d.index_handle = g.index_handle
ORDER BY [Total Cost] DESC;

Note that the results do not specify the order in which the columns in the required index should be created. To determine this, you should inspect your collective SQL code base. As a general rule, the most selected columns should appear first in the index.

I should also point out that only the user columns (such as user\_seeks and user\_scans) are considered in calculating the cost of the missing index. This is because system columns tend to represent using statistics, Database Consistency Checking (DBCC), and Data Definition Language (DDL) commands, and these are less critical to fulfilling the business functionality (as opposed to the database admin functionality).

It is very important for you to remember that you need to take special consideration with regard to the potential cost of the additional index when any data modifications occur in the underlying table. For this reason, additional research into the underlying SQL code base should be undertaken.

If you find that numerous columns are recommended as columns to include, you should examine the underlying SQL since this may indicate that the catchall "SELECT \*" statement is being used—if this turns out to indeed be the case, you should probably revise your select queries.

## **Unused Indexes**

Unused indexes can have a negative impact on performance. This is because when the underlying table data is modified, the index may need to be updated also. This, of course, takes additional time and can even increase the probability of blocking.

When an index is used to satisfy a query and when it is updated as a result of updates applied to the underlying table data, SQL Server updates the corresponding index usage details. These usage details can be viewed to identify any unused indexes.

The sys.dm\_db\_index\_usage\_stats DMV tells how often and to what extent indexes are used. It is joined to the sys.indexes DMV, which contains information used in the creation of the index. You can inspect the various user columns for a value of 0 to identify unused indexes. The impact of the system columns is again ignored for the reasons discussed above. The script shown in Figure 5 will let you identify the most costly unused indexes.

In [7]:
-- Create required table structure only. 
-- Note: this SQL must be the same as in the Database loop given in the following step. 
SELECT
    TOP 1
    DatabaseName = DB_NAME()
     ,TableName = OBJECT_NAME(s.[object_id]) 
     ,IndexName = i.name 
     ,user_updates 
     ,system_updates 
   -- ,*
INTO #TempUnusedIndexes
FROM
    sys.dm_db_index_usage_stats s
    INNER JOIN sys.indexes i ON s.[object_id] = i.[object_id] AND s.index_id = i.index_id
WHERE 
    s.database_id = DB_ID() 
    AND OBJECTPROPERTY(s.[object_id], 'IsMsShipped') = 0
    AND user_seeks = 0 
    AND user_scans = 0 
    AND user_lookups = 0 
    AND s.[object_id] = -999
-- Dummy value to get table structure.
;
-- Loop around all the databases on the server. 
EXEC sp_MSForEachDB 'USE [?]; 
-- Table already exists. 
INSERT INTO #TempUnusedIndexes 
SELECT TOP 100 
    DatabaseName = DB_NAME() ,
    TableName = OBJECT_NAME(s.[object_id]) 
    ,IndexName = i.name 
    ,user_updates 
    ,system_updates 
FROM
     sys.dm_db_index_usage_stats s
    INNER JOIN sys.indexes i ON s.[object_id] = i.[object_id] AND s.index_id = i.index_id 
WHERE 
    s.database_id = DB_ID() 
    AND OBJECTPROPERTY(s.[object_id], ''IsMsShipped'') = 0 
    AND user_seeks = 0
     AND user_scans = 0 
     AND user_lookups = 0 
     AND i.name IS NOT NULL 
-- Ignore HEAP indexes. 
ORDER BY user_updates DESC ; '
-- Select records. 
SELECT
    TOP 100
    *
FROM
    #TempUnusedIndexes
ORDER BY [user_updates] DESC
-- Tidy up.
DROP TABLE #TempUnusedIndexes

(0 rows affected)

The statement has been terminated.

(58 rows affected)

Total execution time: 00:00:05.259

DatabaseName,TableName,IndexName,user_updates,system_updates
CRM,Interacao,ix_tuning_02,782482,0
DBMon,CommandLog,IDX_CommandLog_01,158126,0
Senac_Producao,VW_PENDENCIAS_DIA_TURMA,IDX_VW_PEND_DIA_ALUNO,59918,0
Integrador,Execucao,IX_ConfiguracaoMetodo_Id,55738,0
CRM,Pessoa,IX_DataCadastro,51767,0
Integrador,Execucao,IX_Operador_Id,45296,0
Integracao_BlackBoard,RS_CURSOS_20EAD_LOG,PK_RS_CURSOS_20EADLOG,44696,0
CRM,Pessoa,IX_DataNascimento,42156,0
CRM,Pessoa,IX_Sexo,40776,0
Integracao_BlackBoard,RS_COMPONENTES_LOG,PK_RS_COMPONENTES_LOG,38576,0


: Msg 1222, Level 16, State 56, Line 3
Lock request time out period exceeded.

The results of this query show the indexes that have not been used to retrieve data but have been updated as a result of changes in the underlying table. These updates are shown in the user\_updates and system\_updates columns. The results are sorted by the number of user updates that have been applied to the index.

You must collect enough information to ensure that the index is not used—you don't want to inadvertently remove an index that is perhaps critical for a query that is run only quarterly or annually. Also, note that some indexes are used to constrain the insertion of duplicate records or for ordering of data; these factors must also be considered before removing any unused indexes.

The basic form of the query is applied only to the current database, since it joins to the sys.indexes DMV, which is only concerned with the current database. You can extract results for all the databases on the server using the system stored procedure sp\_MSForEachDB. The pattern I use to do this is explained in the sidebar "Looping through All Databases". I use this pattern in other sections of the script as well where I want to iterate over all the databases on the server. Additionally, I filtered out indexes that are of type heap, since these represent the native structure of a table without a formal index.

## **Costly Used Indexes**

It can also be helpful to identify the indexes (among those that are used) that are most costly in terms of changes to the underlying tables. This cost has a negative impact on performance, but the index itself may be important for data retrieval.

The sys.dm\_db\_index\_usage\_stats DMV lets you see how often and to what extent indexes are used. This DMV is joined to the sys.indexes DMV, which contains details used in the creation of the index. Inspecting the user\_updates and system\_updates columns will show the indexes that are highest maintenance. Figure 6 provides the script used to identify the most costly indexes and shows the results.

In [5]:
-- Create required table structure only. 
-- Note: this SQL must be the same as in the Database loop given in the following step. 
SELECT TOP 1
  [Maintenance cost] = (user_updates + system_updates),
  [Retrieval usage] = (user_seeks + user_scans + user_lookups),
  DatabaseName = DB_NAME(),
  TableName = OBJECT_NAME(s.[object_id]),
  IndexName = i.name 
  INTO #TempMaintenanceCost
FROM sys.dm_db_index_usage_stats s
INNER JOIN sys.indexes i
  ON s.[object_id] = i.[object_id]
  AND s.index_id = i.index_id
WHERE s.database_id = DB_ID()
AND OBJECTPROPERTY(s.[object_id],'IsMsShipped') = 0
AND (user_updates + system_updates) > 0
-- Only report on active rows.
AND s.[object_id] = -999
-- Dummy value to get table structure.
;
-- Loop around all the databases on the server.
EXEC sp_MSForEachDB 'USE [?]; 
-- Table already exists.
INSERT INTO #TempMaintenanceCost SELECT TOP 10 [Maintenance cost] = 
(user_updates + system_updates) ,[Retrieval usage] = (user_seeks + user_scans + user_lookups) ,
DatabaseName = DB_NAME() ,TableName = OBJECT_NAME(s.[object_id]) ,
IndexName = i.name FROM sys.dm_db_index_usage_stats s INNER JOIN sys.indexes i 
ON s.[object_id] = i.[object_id] AND s.index_id = i.index_id WHERE s.database_id = DB_ID() 
AND i.name IS NOT NULL 
-- Ignore HEAP indexes.
AND OBJECTPROPERTY(s.[object_id], ''IsMsShipped'') = 0 
AND (user_updates + system_updates) > 0 
-- Only report on active rows.
ORDER BY [Maintenance cost] DESC ; '
-- Select records.
SELECT TOP 10 *
FROM #TempMaintenanceCost
ORDER BY [Maintenance cost] DESC
-- Tidy up. 
DROP TABLE #TempMaintenanceCost

(0 rows affected)

The statement has been terminated.

(10 rows affected)

Total execution time: 00:00:06.726

Maintenance cost,Retrieval usage,DatabaseName,TableName,IndexName
8648779,9713707,Senac_Producao,REF_ERROSALUNO,PK_REF_ERROSALUNO_1
938123,887117,Integracao_BlackBoard,CAD_INTEGRACAONOTAS,CAD_INTEGRACAONOTAS_PK
781913,6553572,CRM,Interacao,PK_dbo.Interacao
781913,239296,CRM,Interacao,IX_Data
781913,1259027,CRM,Interacao,IX_Interacao
781913,2104971,CRM,Interacao,ix_tuning_03
781913,4770476,CRM,Interacao,ix_tuning_10
781808,0,CRM,Interacao,ix_tuning_02
781808,17607,CRM,Interacao,ix_tuning_01
781808,960496,CRM,Interacao,IX_Oportunidade_Id


: Msg 1222, Level 16, State 56, Line 3
Lock request time out period exceeded.

## **Costly Queries by I/O**

I/O is a measure of the number of reads/writes a query makes. This can be used as an indicator of how efficient a query is—queries that use a lot of I/O are often good subjects for performance improvements.

The sys.dm\_exec\_query\_stats DMV provides aggregate performance statistics for cached query plans, including details about physical and logical reads/writes and the number of times the query has executed. It contains offsets used to extract the actual SQL from its contained parent SQL. This DMV is joined to the sys.dm\_exec\_sql\_text DMF, which contains information about the SQL batch that the I/O relates to. The various offsets are applied to this batch to obtain the underlying individual SQL queries. The script is shown in Figure 9. The results, which are sorted by average I/O, show the average I/O, total I/O, individual query, parent query (if individual query is part of a batch), and the database name.

In [7]:
SELECT TOP 30
  [Average IO] = (total_logical_reads + total_logical_writes) / qs.execution_count,
  [Total IO] = (total_logical_reads + total_logical_writes),
  [Execution count] = qs.execution_count,
  [Individual Query] = SUBSTRING(qt.text, qs.statement_start_offset / 2, (CASE
    WHEN qs.statement_end_offset = -1 THEN LEN(CONVERT(nvarchar(max), qt.text)) * 2
    ELSE qs.statement_end_offset
  END - qs.statement_start_offset) / 2),
  [Parent Query] = qt.text,
  DatabaseName = DB_NAME(qt.dbid)
FROM sys.dm_exec_query_stats qs
CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) AS qt
ORDER BY [Average IO] DESC;

(0 rows affected)

Total execution time: 00:00:07.865

Average IO,Total IO,Execution count,Individual Query,Parent Query,DatabaseName


## **Costly Queries by CPU**

Another rather useful approach that you can take is to analyze the most costly queries in terms of CPU usage. This approach can be very indicative of queries that are performing poorly. The DMVs that I will use here are the same as the ones I just used for examining queries in terms of I/O. The query that you see in Figure 10 allows you to identify the most costly queries as measured by CPU usage.

In [5]:
SELECT
    TOP 20
    [Average CPU used] = total_worker_time / qs.execution_count
    ,[Total CPU used] = total_worker_time
    ,[Execution count] = qs.execution_count
    ,[Individual Query] = SUBSTRING(qt.text, qs.statement_start_offset / 2, (CASE
    WHEN qs.statement_end_offset = -1 THEN LEN(CONVERT(NVARCHAR(max), qt.text)) * 2
    ELSE qs.statement_end_offset
  END - qs.statement_start_offset) / 2)
    ,[Parent Query] = qt.text
    ,DatabaseName = DB_NAME(qt.dbid)
FROM
    sys.dm_exec_query_stats qs
CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) AS qt
ORDER BY [Average CPU used] DESC;